In [1]:
# -----------------------------
# Gerar dataset sintético balanceado + métricas finais
# -----------------------------
import os
import pandas as pd
import numpy as np

# -----------------------------
# 1. Configuração
# -----------------------------
num_nodes = 100
edges_per_class = 200   # número de arestas por classe
rng = np.random.default_rng(42)

# features dos nós
X = rng.normal(size=(num_nodes, 3))

# -----------------------------
# 2. Função de geração por classe
# -----------------------------
def generate_edges_for_class(label, n_edges, num_nodes, rng, X):
    edges, latencies, labels = [], [], []
    for _ in range(n_edges):
        i, j = rng.choice(num_nodes, size=2, replace=False)

        if label == "baixa":
            latency = rng.uniform(10, 49)   # < 50ms
        elif label == "media":
            latency = rng.uniform(60, 149)  # [50,150)
        else:
            latency = rng.uniform(160, 300) # >= 150ms

        edges.append((i, j))
        latencies.append(latency)
        labels.append(label)
    return edges, latencies, labels

# -----------------------------
# 3. Geração balanceada
# -----------------------------
edges, latencies, labels = [], [], []
for label in ["baixa", "media", "alta"]:
    e, l, lab = generate_edges_for_class(label, edges_per_class, num_nodes, rng, X)
    edges.extend(e)
    latencies.extend(l)
    labels.extend(lab)

# -----------------------------
# 4. Criar DataFrames
# -----------------------------
nodes_df = pd.DataFrame(X, columns=["feat_1","feat_2","feat_3"])
nodes_df.index.name = "node_id"

edges_df = pd.DataFrame({
    "src": [i for (i,_) in edges],
    "dst": [j for (_,j) in edges],
    "latency_ms": latencies,
    "label": labels
})

# -----------------------------
# 5. Salvar dataset (mesmos nomes)
# -----------------------------
os.makedirs("../datasets", exist_ok=True)
nodes_path = "../datasets/radnet_synthetic_nodes.csv"
edges_path = "../datasets/radnet_synthetic_edges.csv"
nodes_df.to_csv(nodes_path)
edges_df.to_csv(edges_path, index=False)

# -----------------------------
# 6. Mostrar métricas finais
# -----------------------------
print("Prévia — Nós (features):")
print(nodes_df.head().to_string(index=True))
print("\nPrévia — Arestas (latência e rótulos):")
print(edges_df.head().to_string(index=False))

print("\nDistribuição das classes (label):")
print(edges_df["label"].value_counts())

print("\nArquivos gerados:")
print(f"- Nós: {nodes_path}")
print(f"- Arestas: {edges_path}")

Prévia — Nós (features):
           feat_1    feat_2    feat_3
node_id                              
0        0.304717 -1.039984  0.750451
1        0.940565 -1.951035 -1.302180
2        0.127840 -0.316243 -0.016801
3       -0.853044  0.879398  0.777792
4        0.066031  1.127241  0.467509

Prévia — Arestas (latência e rótulos):
 src  dst  latency_ms label
  23   59   24.289306 baixa
  73   36   24.799099 baixa
  68   18   47.005459 baixa
  29   67   28.755507 baixa
  32   76   43.093859 baixa

Distribuição das classes (label):
label
baixa    200
media    200
alta     200
Name: count, dtype: int64

Arquivos gerados:
- Nós: ../datasets/radnet_synthetic_nodes.csv
- Arestas: ../datasets/radnet_synthetic_edges.csv
